<a href="https://colab.research.google.com/github/Nuzhattttt/braintumor/blob/main/P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_directory = '/content/gdrive/MyDrive/Amar dataset/dataset/Training'
test_directory = '/content/gdrive/MyDrive/Amar dataset/dataset/Testing'

In [5]:
image_size = (128, 128)
train_images = []
train_labels = []
test_images = []
test_labels = []

In [6]:
def load_and_preprocess_images(directory, images_list, labels_list):
    for class_name in os.listdir(directory):
        class_directory = os.path.join(directory, class_name)

        for filename in os.listdir(class_directory):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(class_directory, filename)
                image = Image.open(image_path).convert("RGB")
                image = image.resize(image_size, Image.LANCZOS)
                images_list.append(image)
                labels_list.append(class_name)


In [30]:
import numpy as np
import random
import os
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
def create_clients(data_list, label_list, num_clients=10, initial='clients'):
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
    data = list(zip(data_list, label_list))
    random.shuffle(data)
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]
    assert(len(shards) == len(client_names))
    return {client_names[i] : shards[i] for i in range(len(client_names))}
def batch_data(data_shard, bs=32):
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(500,input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(300))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("sigmoid"))
        return model
class VGG16:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Conv2D(input_shape=shape,filters=32,kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))


        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Flatten())
        model.add(Dense(units=4096,activation="relu"))
        model.add(Dense(units=4096,activation="relu"))
        model.add(Dense(units=classes, activation="softmax"))
        return model

def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

def scale_model_weights(weight, scalar):
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    avg_grad = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad

def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    predictions = model(X_test)
    loss = cce(Y_test, predictions)
    acc = accuracy_score(tf.argmax(Y_test, axis=1), tf.argmax(predictions, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [31]:
load_and_preprocess_images(train_directory, train_images, train_labels)


In [32]:
load_and_preprocess_images(test_directory, test_images, test_labels)


In [33]:
print(test_labels)

['pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary', 'pituitary'

In [34]:
label_encoder = LabelEncoder()
train_encoded_labels = label_encoder.fit_transform(train_labels)
test_encoded_labels = label_encoder.transform(test_labels)

In [35]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [36]:
X_train = np.array([np.array(img) for img in train_images])
y_train = np.array(train_encoded_labels)

In [37]:
print(len(X_train))

11444


In [38]:
augmented_images = []
for image in X_train:
    augmented_image = datagen.random_transform(image)
    augmented_images.append(augmented_image)
X_train_augmented = np.array(augmented_images)

In [39]:
print(X_train_augmented.shape)

(11444, 128, 128, 3)


In [40]:
X_test = np.array([np.array(img) for img in test_images])
y_test = np.array(test_encoded_labels)


In [41]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (11444, 128, 128, 3)
y_train shape: (11444,)
X_test shape: (2622, 128, 128, 3)
y_test shape: (2622,)


In [42]:
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

labels = list(set(train_encoded_labels.tolist()))
#lb = LabelBinarizer()
#label_list = lb.fit_transform(label_list)
n_values = np.max(train_encoded_labels) + 1
train_encoded_labels = np.eye(n_values)[train_encoded_labels]

X_train, X_test, y_train, y_test = train_test_split(X_train,
                                                    train_encoded_labels,
                                                    test_size=0.1,
                                                    random_state=42)

clients = create_clients(X_train, y_train, num_clients=10, initial='client')


In [43]:
print(y_test)

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]


In [44]:
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))
print(test_batched)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>


In [45]:
print(clients_batched)

{'client_1': <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>, 'client_2': <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>, 'client_3': <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>, 'client_4': <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>, 'client_5': <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>, 'client_6': <_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))

In [46]:
len(labels)

4

In [47]:
from tensorflow.keras.optimizers.legacy import SGD
comms_round = 10
#create optimizer
lr = 0.01
loss='categorical_crossentropy'
metrics = ['accuracy']
# optimizer = Adam(learning_rate=lr,
#                 weight_decay=lr / comms_round,
#                 ema_momentum=0.9
#                )
optimizer = SGD(lr=lr,
                decay=lr / comms_round,
                momentum=0.9
               )


#initialize global model
#print(data_list.shape,labels)
# smlp_global = SimpleMLP()
# global_model = smlp_global.build(data_list[0].shape,len(labels))

smpl_vgg = VGG16()
global_model = smpl_vgg.build(X_train[0].shape,len(labels))

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [48]:
print(global_model.get_weights())

[array([[[[-0.10172668, -0.11338519, -0.01859817, -0.01272939,
          -0.07959884, -0.07603893, -0.08626823,  0.02580279,
           0.07585205,  0.01552384,  0.00036465,  0.04178473,
           0.04599561,  0.04470353, -0.07231172,  0.09050512,
           0.12440409, -0.12306712, -0.00685149, -0.07296699,
          -0.11179262,  0.11620803,  0.06312303,  0.05102351,
          -0.12498119,  0.03657158,  0.09395416, -0.07420016,
          -0.02233785, -0.08430552, -0.01755262, -0.01744242],
         [ 0.02553192,  0.03707528, -0.13367878,  0.00514971,
          -0.04821406,  0.09529312,  0.01284492,  0.0552281 ,
           0.02054775,  0.10945696, -0.0192256 , -0.09613861,
           0.11384489,  0.08033127, -0.09231292,  0.11881928,
           0.12348451,  0.11677389, -0.13417824, -0.04073224,
           0.08965407, -0.08339228, -0.03695354,  0.05839664,
          -0.03610818,  0.10201648, -0.02694584, -0.03325618,
           0.06982625, -0.07316866,  0.05291955, -0.0634813 ],
     

In [50]:
for comm_round in range(comms_round):
    global_weights = global_model.get_weights()
    scaled_local_weight_list = list()
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    for client in tqdm(client_names , desc = 'Progress Bar'):
        #time.sleep(0.5)
        smpl_local = VGG16()
        local_model = smpl_vgg.build(X_train[0].shape,len(labels))
        smlp_local = SimpleMLP()
        local_model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)
        #print(local_model.summary())
        #print(clients_batched)
        local_model.set_weights(global_weights)
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        K.clear_session()
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    global_model.set_weights(average_weights)
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
        batch_size=32
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(batch_size)
smlp_SGD = VGG16()
# SGD_model = smlp_SGD.build(X_train[0].shape,len(labels))
n_classes_for_testing = min(n_classes, 10)
SGD_model = smpl_vgg.build(X_train[0].shape, n_classes_for_testing)
SGD_model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)


Progress Bar:   0%|          | 0/10 [00:10<?, ?it/s]


ResourceExhaustedError: ignored